<a href="https://colab.research.google.com/github/emazakaria/MyFinalYearProject/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dash pandas plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.4 MB/s eta 0:00:00


In [ ]:
!pip install opencage

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from opencage.geocoder import OpenCageGeocode

<ipython-input-3-351192220ff3>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-351192220ff3>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [ ]:
# Read the dataset
df = pd.read_csv('stigma_coords.csv', encoding='unicode_escape')

df.shape

(502, 6)

In [ ]:
# Your OpenCage API key
api_key = '05699016136d4744bd9f3fb40d4cbfa7'

# Initialize the geocoder
geocoder = OpenCageGeocode(api_key)

# Function to get country from location using geocoding
def get_country_from_location(location):
    results = geocoder.geocode(location)
    if results and 'components' in results[0]:
        country = results[0]['components'].get('country')
        return country
    return None

# Apply the function to create a new 'country' column
df['country'] = df['location'].apply(get_country_from_location)

In [ ]:
df.head(100)

,Date,location,tweet,predicted_stigma,latitude,longitude,country
0,12/10/2022,UK,as its id urge anyone to try and be a bit more...,1,54.702354,-3.276575,United Kingdom
1,14/10/2021,1884 Market St San Francisco,this week is ocd awareness week is not all abo...,1,37.771059,-122.424836,United States
2,14/10/2021,"San Francisco, CA",this week is ocd awareness week is not all abo...,1,37.779026,-122.419906,United States
3,12/10/2021,"Warren, Ohio",youve probably heard someone say im sooooo ocd...,1,39.420398,-84.180897,United States
4,16/10/2020,"gorseinon, wales, uk",ocd is horrific to live with and while creatin...,1,51.669380,-4.041592,United Kingdom
...,...,...,...,...,...,...,...
95,17/5/2018,"London, England",are you,1,51.507446,-0.127765,United Kingdom
96,17/5/2018,Broad City,this i want to reinforce the inherent worth of...,1,51.519304,-0.082801,United Kingdom
97,17/5/2018,London,is your brain a machine made up of billions of...,1,51.507446,-0.127765,United Kingdom
98,17/5/2018,"London, England",i was convinced i was dying but didnt ring an ...,1,51.507446,-0.127765,United Kingdom


In [ ]:
# Group by country and calculate stigma count
stigma_counts = df.groupby('country')['predicted_stigma'].sum().reset_index()

# Create a choropleth map
choropleth_fig = px.choropleth(
    stigma_counts,
    locations='country',
    locationmode="country names",
    color='predicted_stigma',
    color_continuous_scale='bluered',
    title='Stigma Count by Country',
    labels={'predicted_stigma': 'Stigma Count'},
)


# Create a scatter plot to show the predicted stigma count for each country
scatter_fig = px.scatter(
    stigma_counts,
    x='country',
    y='predicted_stigma',
    title='Predicted Stigma Count by Country',
    labels={'predicted_stigma': 'Stigma Count'},
)



# # Create a heatmap
# heatmap_fig = px.density_heatmap(
#     df,
#     x='longitude',
#     y='latitude',
#     title='Stigma Density Heatmap',
#     labels={'predicted_stigma': 'Stigma Count'},
# )

# Create a bar chart to show the predicted stigma count for each country
bar_fig = px.bar(
    stigma_counts,
    x='country',
    y='predicted_stigma',
    title='Predicted Stigma Count by Country',
    labels={'predicted_stigma': 'Stigma Count'},
)


# Create a bubble map to visualize stigma predictions based on the count, with bubble sizes indicating the intensity
bubble_fig = px.scatter_geo(
    df,
    lat='latitude',
    lon='longitude',
    color='predicted_stigma',
    size='predicted_stigma',
    title='Stigma Predictions Bubble Map',
    labels={'predicted_stigma': 'Stigma Count'},
)

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Stigma Dashboard"),

    dcc.Graph(figure=choropleth_fig),
    dcc.Graph(figure=scatter_fig),
    # dcc.Graph(figure=time_series_fig),
    dcc.Graph(figure=bar_fig),
    dcc.Graph(figure=bubble_fig),
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>